In [1]:
import pandas as pd
from tqdm import tqdm

In [2]:
pd.set_option('display.max_colwidth', None)

In [3]:
irecommend = pd.read_csv("../data/raw/irecommend.csv")
somelie = pd.read_csv("../data/raw/somelie.csv")
vinofan = pd.read_csv("../data/raw/vinofan.csv")

In [4]:
concat = pd.concat([irecommend, vinofan, somelie])

In [5]:
concat["wine_name"] = concat["wine_name"].map(lambda x: x.replace(" ", " "))
concat.head(3)

,wine_name,username,rating,variants_number,wine_type,brand,wine_link,review_link,Unnamed: 0
0,Вино столовое полусладкое розовое ООО «Атанель» «Крымское»,Ekaterina_ssss,4,5.0,NaN,ООО «Атанель»,NaN,https://irecommend.ru/content/vino-ooo-atanel-vino-stolovoe-polusladkoe-rozovoe-krymskoe,NaN
1,Вино Fanagoria Столовое десертное красное ICE WINE (ледяное вино) Saperavi,Marish@❤️,5,5.0,NaN,Fanagoria,NaN,https://irecommend.ru/content/vino-fanagoria-stolovoe-desertnoe-krasnoe-ice-wine-ledyanoe-vino-saperavi,NaN
2,Вино Фанагория Кагор канонический NR номерной резерв 1957,Вера-Верочка,4,5.0,NaN,Фанагория,NaN,https://irecommend.ru/content/vino-fanagoriya-kagor-kanonicheskii-nr-nomernoi-rezerv-1957,NaN


In [6]:
from collections import Counter
s = "".join(filter(lambda x: x not in "«»()/\"'%!-–&.,1234567890", " ".join(concat["wine_name"].drop_duplicates()).lower()))
words = list(filter(lambda x: len(x) >= 5 and x not in ["вино", "де", "игристое", "вина",
                                       "ди", "ооо", "специальное", "псладкое", "natural"
                                                       ], s.split()))
Counter(words).most_common(10000)

[('красное', 134),
 ('полусладкое', 112),
 ('белое', 90),
 ('сухое', 76),
 ('столовое', 75),
 ('мерло', 64),
 ('совиньон', 59),
 ('sauvignon', 59),
 ('каберне', 57),
 ('шардоне', 51),
 ('cabernet', 51),
 ('merlot', 47),
 ('резерва', 47),
 ('массандра', 46),
 ('chianti', 46),
 ('кагор', 43),
 ('chateau', 40),
 ('reserva', 40),
 ('кьянти', 38),
 ('blanc', 33),
 ('розовое', 32),
 ('rosso', 30),
 ('pinot', 30),
 ('классико', 30),
 ('chardonnay', 29),
 ('classico', 27),
 ('мускат', 25),
 ('полусухое', 25),
 ('crianza', 24),
 ('резерв', 23),
 ('долина', 23),
 ('саперави', 22),
 ('фанагория', 21),
 ('bianco', 20),
 ('tempranillo', 19),
 ('инкерман', 18),
 ('villa', 17),
 ('valley', 17),
 ('санта', 17),
 ('мадера', 16),
 ('херес', 16),
 ('baron', 16),
 ('вьехо', 16),
 ('крианца', 16),
 ('lambrusco', 15),
 ('вилла', 15),
 ('портвейн', 15),
 ('riserva', 15),
 ('россо', 15),
 ('ризерва', 15),
 ('темпранильо', 15),
 ('кубань', 14),
 ('tamagne', 14),
 ('тамань', 14),
 ('алазанская', 14),
 ('крымски

In [7]:
irecommend_wines_original = irecommend["wine_name"].drop_duplicates()
somelie_wines_original = somelie["wine_name"].drop_duplicates()
vinofan_wines_original = vinofan["wine_name"].drop_duplicates()

In [8]:
def f(s):
    s = "".join(filter(lambda x: x not in "«»()/\"'%!-–&.,1234567890", s))
    s = s.lower()
    to_delete = ["вино", "шампанское", 'красное', 'полусладкое', 'белое', 'сухое', 'столовое', 'мерло', 'совиньон', 'sauvignon', 'каберне', 'шардоне', 'cabernet', 'merlot', 'резерва', 'массандра', 'chianti', 'кагор', 'chateau', 'reserva', 'кьянти', 'blanc', 'розовое', 'rosso', 'pinot', 'классико', 'chardonnay', 'classico', 'мускат', 'полусухое', 'crianza', 'резерв', 'rose', 'del', 'di', 'де', 'de', 'бордо', 'la', 'дю', 'ди', 'кавеш', 'санта', 'месяцев', 'лиму', 'креман',]
    for word in to_delete:
        s = s.replace(word, "")
    s = s.replace("\xa0", " ")
    s = s.strip()
    s = " ".join(s.split())
    return s

irecommend_wines = irecommend_wines_original.map(f)
somelie_wines = somelie_wines_original.map(f)
vinofan_wines = vinofan_wines_original.map(f)

for wine_i, wine_i_original in zip(irecommend_wines, irecommend_wines_original):
    max_j, max_set = "", set()
    for wine_j, wine_j_original in zip(somelie_wines, somelie_wines_original):
        common = set(wine_i.split()).intersection(set(wine_j.split()))
        if len(common) > 2 and len(common) > len(max_set):
            max_set = common
            max_j = wine_j_original
    if len(max_set) != 0:
        print(f"[\"{wine_i_original}\", \"{max_j}\"],")

print()

for wine_i, wine_i_original in zip(irecommend_wines, irecommend_wines_original):
    max_j, max_set = "", set()
    for wine_j, wine_j_original in zip(vinofan_wines, vinofan_wines_original):
        common = set(wine_i.split()).intersection(set(wine_j.split()))
        if len(common) > 2 and len(common) > len(max_set):
            max_set = common
            max_j = wine_j_original
    if len(max_set) != 0:
        print(f"[\"{wine_i_original}\", \"{max_j}\"],")

print()
        
for wine_i, wine_i_original in zip(somelie_wines, somelie_wines_original):
    max_j, max_set = "", set()
    for wine_j, wine_j_original in zip(vinofan_wines, vinofan_wines_original):
        common = set(wine_i.split()).intersection(set(wine_j.split()))
        if len(common) > 2 and len(common) > len(max_set):
            max_set = common
            max_j = wine_j_original
    if len(max_set) != 0:
        print(f"[\"{wine_i_original}\", \"{max_j}\"],")




["Вино ООО "Кубань - вино" Мадера Кубанская Chateau Tamagne", "Вино Madera Chateau Tamagne Reserve  Мадера Кубанская Шато Тамань Резерв"],
["Вино Шато Тамань сухое белое Fleurs du Sud", "Вино Rose de Tamagne Fleurs du Sud  Роза Тамани Шато Тамань"],
["Вино Concha y Toro "Casillero Del Diablo Cabernet Sauvignon" (Казильеро Дель Дьябло Каберне Савиньон) алк. 13,5% красное сухое 0,75л ", "Вино Casillero del Diablo Cabernet Sauvignon Reserva  Казильеро дель Дьябло Каберне Совиньон Резерва"],
["Вино Массандра Портвейн красный крымский", "Вино Crimsky Red Port Massandra  Портвейн Красный Крымский Массандра"],
["Вино  Cusumano Nero D'Avola", "Вино Cusumano Nero d'Avola Terre Siciliane  Кузумано Неро д'Авола Терре Сичилиане"],
["Вино ООО "Кубань - вино"  столовое полусухое розовое Роза Тамани. Шато Тамань", "Вино Rose de Tamagne Fleurs du Sud  Роза Тамани Шато Тамань"],
["Вино Plantaze Vranac Pro Corde", "Вино Plantaze Vranac Pro Corde  Плантаже Вранац Про Корде"],
["Вино Мадера дионис крымск

In [9]:
coincidences = [
    ["Вино ООО \"Кубань - вино\" Мадера Кубанская Chateau Tamagne", "Вино Madera Chateau Tamagne Reserve  Мадера Кубанская Шато Тамань Резерв"],
    ["Вино Шато Тамань сухое белое Fleurs du Sud", "Вино Rose de Tamagne Fleurs du Sud  Роза Тамани Шато Тамань"],
    ["Вино Concha y Toro \"Casillero Del Diablo Cabernet Sauvignon\" (Казильеро Дель Дьябло Каберне Савиньон) алк. 13,5% красное сухое 0,75л ", "Вино Casillero del Diablo Cabernet Sauvignon Reserva  Казильеро дель Дьябло Каберне Совиньон Резерва"],
    ["Вино Массандра Портвейн красный крымский", "Вино Crimsky Red Port Massandra  Портвейн Красный Крымский Массандра"],
    ["Вино  Cusumano Nero D'Avola", "Вино Cusumano Nero d'Avola Terre Siciliane  Кузумано Неро д'Авола Терре Сичилиане"],
    ["Вино ООО \"Кубань - вино\"  столовое полусухое розовое Роза Тамани. Шато Тамань", "Вино Rose de Tamagne Fleurs du Sud  Роза Тамани Шато Тамань"],
    ["Вино Plantaze Vranac Pro Corde", "Вино Plantaze Vranac Pro Corde  Плантаже Вранац Про Корде"],
    ["Вино Мадера дионис крымская Старый Крым", "Вино Madera Stary Crym Dionis  Мадера Дионис Крымская Старый Крым"],
    ["Вино Freixenet Carta Nevada Dulce Фрешенед Кава Карта Невада Дульче игристое белое сладкое", "Вино Freixenet Cava Carta Nevada Semi Seco  Фрешенет Кава Карта Невада"],
    ["Вино Concha y Toro  VCT Casillero del Diablo Shiraz", "Вино Casillero del Diablo Shiraz Reserva  Казильеро дель Дьябло Шираз Резерва"],
    ["Вино Casa Vitivinicola Tinazzi POGGIO AI SANTI Rosato", "Вино Poggio ai Santi Sangiovese  Поджио ай Санти Санджовезе"],
    ["Вино ООО \"Кубань - вино\" Chateau Tamagne DUO Rose (Шато Тамань Дуо Розовое)", "Вино Cabernet Merlot Chateau Tamagne Duo  Каберне Мерло Шато Тамань Дуо"],
    ["Вино   Jerez Fino Romate (Херес Фино Ромате)", "Вино Osborne Sherry Fino  Осборн Херес Фино"],
    ["Вино   Friuli Venezia Giulia - Villa Dragoni Friuli Grave Refosco dal peduncolo rosso -Emmegio тм", "Вино Moletto Refosco dal Peduncolo Rosso  Молетто Рефоско даль Педунколо Россо"],
    ["Вино Bodegas Penalba Lopez / Бодегас Пеньялба Лопез Monte Castrillo Tinto\Монте Кастрильо", "Вино Penalba Lopez Cava Brut Nature  Пеньялба Лопез Кава Брют Натур"],
    ["Вино Concha y Toro Casillero del Diablo Carmenere Reserva, 2010", "Вино Casillero del Diablo Carmenere Reserva  Казильеро дель Дьябло Карменер Резерва"],
    ["Вино   SICILIA - Cusumano Nadaria Nero d'Avola I.G.T.", "Вино Cusumano Nero d'Avola Terre Siciliane  Кузумано Неро д'Авола Терре Сичилиане"], 
    ["Вино Pago de Tharsys Cava Brut Rosado  Паго де Тарсис Кава Брют Розовое", "вино игристое КАВА ДОМИНИО ДЕ ТАРСИС БРЮТ НАТЮР 11.5% 0.75"],
    ["Вино Burgo Viejo Garnacha  Бурго Вьехо Гарнача", "вино БУРГО ВЬЕХО ГАРНАЧА 13.5% 0.75"],
    ["Вино Burgo Viejo Graciano  Бурго Вьехо Грасиано", "вино БУРГО ВЬЕХО ГРАСИАНО 14% 0.75"],
    ["Вино Morande Pinot Noir Gran Reserva  Моранде Пино Нуар Гран Резерва", "вино МОРАНДЕ ГРАН РЕЗЕРВА ПИНО НУАР 13.5-14% 0.75"],
    ["Вино Pago de Tharsys Cava Brut Nature  Паго де Тарсис Кава Брют Натюр", "вино игристое КАВА ДОМИНИО ДЕ ТАРСИС БРЮТ НАТЮР 11.5% 0.75"],
    ["Вино Luigi Leonardo Pinot Grigio  Луиджи Леонардо Пино Гриджио", "вино ЛУИДЖИ ЛЕОНАРДО ПИНО ГРИДЖИО 12% 0,75"],
    ["Вино Eidosela Seleccion Albarino  Эйдосела Селексьон Альбариньо", "вино ЭЙДОСЕЛА СЕЛЕКСЬОН АЛЬБАРИНЬО 12-15% 0.75"],
    ["Вино Luigi Leonardo Montepulciano d'Abruzzo  Луиджи Леонардо Монтепульчано д'Абруццо", "вино ЛУИДЖИ ЛЕОНАРДО МОНТЕПУЛЬЧАНО Д АБРУЦЦО 13% 0,75"],
    ["Вино Burgo Viejo Rosado  Бурго Вьехо Росадо", "вино БУРГО ВЬЕХО РОСАДО 12-14% 0,75"],
    ["Вино Corte Olivi Valpolicella Classico  Корте Оливи Вальполичелла Классико", "вино КОРТЕ ОЛИВИ ВАЛЬПОЛИЧЕЛЛА КЛАССИКО 12.5-13% 0.75"],
    ["Вино Corte Olivi Chiaretto Bardolino Classico  Корте Оливи Кьяретто Бардолино Классико", "вино КЬЯРЕТТО БАРДОЛИНО КЛАССИКО КОРТЕ ОЛИВИ 12% 0,75"],
    ["Вино LFE Pinot Noir Family Selection Gran Reserva  ЛФЭ Пино Нуар Фэмили Селекшн Гран Резерва", "вино МОРАНДЕ ГРАН РЕЗЕРВА ПИНО НУАР 13.5-14% 0.75"],
    ["Вино Zonin Pinot Grigio Delle Venezie  Зонин Пино Гриджо Делле Венецие", "вино ПИНО ГРИДЖИО ДЕЛЛЕ ВЕНЕЦИЕ 12% 0,75"],
    ["Вино Chateau Maison Blanche Cru Bourgeous  Шато Мезон Бланш Крю Буржуа", "вино ШАТО ПЛАНТЕ КРЮ БУРЖУА 13% 0,75"],
    ["Вино Les Dorees Bourgogne Pinot Noir  Ле Доре Бургонь Пино Нуар", "Вино БУРГОНЬ ПИНО НУАР ПРЕСТИЖ 2017 12.5% 0.75"],
    ["Вино Leyenda Pedro Ximenez Sherry  Леенда Педро Хименес Херес", "херес ЛЕЕНДА ПЕДРО ХИМЕНЕС 17% 0,75"],
    ["Вино Chateau Preuillac Cru Bourgeous  Шато Прюйяк Крю Буржуа", "вино ШАТО ПЛАНТЕ КРЮ БУРЖУА 13% 0,75"],
    ["Вино Leyenda Fino Sherry  Леенда Фино Херес", "херес ЛЕЕНДА ФИНО 15% 0,75"],
    ["Вино Leyenda Cream Sherry  Леенда Крим Херес", "херес ЛЕЕНДА КРИМ 17,5% 0,75"],
    ["Вино Chateau Le Monteil d'Arsac Cru Bourgeois  Шато Ле Монтей д'Арсак Крю Буржуа ", "вино ШАТО ПЛАНТЕ КРЮ БУРЖУА 13% 0,75"],
]